In [1]:
import tensorflow as tf
import numpy as np

In [18]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 1s 0us/step


In [92]:
X_train.shape

(60000, 28, 28)

In [ ]:
# def prune(layer1, layer2):
#     pass

# class SSRegularizer(tf.keras.regularizers.Regularizer):
#     def __init__(self, l1):
#         self.l1 = l1

#     def __call__(self, x):
#         scaling_matrix = tf.cumsum(tf.constant(self.l1, shape=x.shape), axis=1) - self.l1
#         return self.l1 * tf.reduce_sum(tf.multiply(scaling_matrix, tf.abs(x)))

#     def get_config(self):
#         return {'l1': float(self.l1)}


# class SSDense(tf.keras.layers.Layer):
#     def __init__(self, units, activation=None, kernel_initializer='glorot_uniform', bias_initializer='zeros', l1=0.1):
#         super().__init__()
#         self.units = units
#         self.activation = tf.keras.activations.get(activation)
#         self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
#         self.bias_initializer = tf.keras.initializers.get(bias_initializer)
#         self.kernel_regularizer = SSRegularizer(l1=l1)

#     def build(self, input_shape):
#         self.kernel = self.add_weight(
#             'kernel',
#             shape=(input_shape[-1], self.units),
#             dtype='float32',
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer)
        
# #         self.bias = self.add_weight(
# #             'bias',
# #             shape=(self.units,),
# #             dtype='float32',
# #             initializer=self.bias_initializer)

#     def call(self, inputs):
# #         return self.activation(tf.matmul(inputs, self.kernel) + self.bias)
#         return self.activation(tf.matmul(inputs, self.kernel))

In [279]:
def prune(layer1, layer2):
    pass

class SSRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, l1):
        self.l1 = l1

    def __call__(self, x):
        scaling_matrix = tf.cumsum(tf.constant(self.l1, shape=x.shape), axis=1) - self.l1
        return self.l1 * tf.reduce_sum(tf.multiply(scaling_matrix, tf.abs(x)))

    def get_config(self):
        return {'l1': float(self.l1)}
    
def ss_regularizer(x):
    scaling_matrix = tf.cumsum(tf.constant(0.1, shape=x.shape), axis=1) - 0.1
    return tf.reduce_sum(tf.multiply(scaling_matrix, tf.abs(x)))


class SSDense(tf.keras.layers.Layer):
    def __init__(self, units, activation=None, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None):
        super().__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        
        self.l1 = 0.000000001
        
#         self.kernel_regularizer = SSRegularizer(l1=l1)    

    def build(self, input_shape):
        self.kernel = tf.Variable(
            name='kernel',
            initial_value=self.kernel_initializer(shape=(input_shape[-1], self.units), dtype='float32'),
            trainable=True)
        
#         self.bias = tf.Variable(
#             name='bias',
#             initial_value=self.bias_initializer(shape=(self.units,), dtype='float32'),
#             trainable=True)

#         super().build(input_shape)

    def call(self, inputs):
#         return self.activation(tf.matmul(inputs, self.kernel) + self.bias)
        print("CALLED")

        if self.kernel_regularizer is not None:
            model.add_loss(lambda: self.kernel_regularizer(self.kernel))

        return self.activation(tf.matmul(inputs, self.kernel))

In [280]:
input_layer = tf.keras.layers.Input(shape=[28, 28])
hidden = SSDense(50, activation='relu', kernel_regularizer=SSRegularizer(l1=0.1))
last = SSDense(10, activation='softmax')

x = tf.keras.layers.Flatten()(input_layer)
x = hidden(x)
output = last(x)

model = tf.keras.Model(inputs=[input_layer], outputs=[output])

CALLED
CALLED


In [281]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics='accuracy')

In [282]:
model.fit(X_train, y_train, epochs=3, validation_data=((X_test), (y_test)))

Epoch 1/3
CALLED
CALLED
CALLED
CALLED
1861/1875 [============================>.] - ETA: 0s - loss: 122.4724 - accuracy: 0.5990CALLED
CALLED
1875/1875 [==============================] - 2s 1ms/step - loss: 121.7919 - accuracy: 0.5996 - val_loss: 9.3072 - val_accuracy: 0.7946
Epoch 2/3
1875/1875 [==============================] - 2s 1ms/step - loss: 6.5386 - accuracy: 0.7597 - val_loss: 7.5185 - val_accuracy: 0.7626
Epoch 3/3
1875/1875 [==============================] - 2s 1ms/step - loss: 4.7312 - accuracy: 0.7938 - val_loss: 5.9147 - val_accuracy: 0.8048


In [275]:
hidden.kernel

<tf.Variable 'ss_dense_83/kernel:0' shape=(784, 50) dtype=float32, numpy=
array([[ 2.80314013e-02,  1.57379662e-04,  2.58008076e-05, ...,
        -2.73253900e-05, -8.19038469e-05, -4.84327029e-04],
       [-5.21218404e-02, -8.68051939e-05,  1.08333988e-04, ...,
        -1.46389633e-04, -9.51471302e-05, -1.00910038e-05],
       [ 6.49969801e-02,  1.02194266e-04, -1.15970630e-04, ...,
         1.98095673e-04, -5.42014459e-06, -7.82775533e-05],
       ...,
       [ 7.36590996e-02, -7.95564556e-05,  7.88620673e-05, ...,
         2.58172862e-04, -2.98755855e-04, -1.36765011e-05],
       [ 7.79336467e-02,  1.24036451e-04,  2.19142195e-04, ...,
        -1.55576825e-04, -7.95407250e-05,  1.73455774e-05],
       [ 6.40873536e-02,  1.28635002e-04,  1.26767962e-04, ...,
        -1.46068822e-04,  1.47869374e-04,  1.14488015e-04]], dtype=float32)>

In [267]:
hidden.kernel_regularizer = SSRegularizer(l1=0.1)

In [283]:
model.fit(X_train, y_train, epochs=2, validation_data=((X_test), (y_test)))

Epoch 1/2
1875/1875 [==============================] - 2s 1ms/step - loss: 3.8699 - accuracy: 0.8207 - val_loss: 5.6891 - val_accuracy: 0.8494
Epoch 2/2
1875/1875 [==============================] - 2s 1ms/step - loss: 3.6533 - accuracy: 0.8395 - val_loss: 5.2297 - val_accuracy: 0.8131


In [278]:
np.amax(hidden.kernel, 0)

array([0.28975675, 0.00046396, 0.00229231, 0.02302813, 0.01906175,
       0.0152954 , 0.01477466, 0.00104287, 0.01882115, 0.00067823,
       0.01305784, 0.00046348, 0.00049716, 0.00053462, 0.00703956,
       0.00069375, 0.00090892, 0.00051005, 0.00075462, 0.00053977,
       0.00043511, 0.00179379, 0.00057123, 0.00067489, 0.00060877,
       0.00059327, 0.00044425, 0.00053964, 0.00055375, 0.00099119,
       0.00073646, 0.00050139, 0.0005046 , 0.00069769, 0.00075718,
       0.00087944, 0.00073467, 0.00057799, 0.00051896, 0.00048253,
       0.00052509, 0.00076614, 0.00053804, 0.00048664, 0.00059923,
       0.00058953, 0.00059621, 0.00062028, 0.00064497, 0.0005404 ],
      dtype=float32)

In [172]:
hidden.kernel.shape

TensorShape([784, 50])

In [173]:
last.kernel.shape

TensorShape([50, 10])

In [128]:
hidden.kernel

<tf.Variable 'ss_dense_13/kernel:0' shape=(784, 50) dtype=float32, numpy=
array([[ 2.19782889e-02, -8.38522101e-06,  1.75410969e-06, ...,
        -2.57352440e-05, -9.40302416e-05, -3.54066688e-05],
       [ 4.62642983e-02, -3.22027790e-06, -1.28006614e-05, ...,
        -1.63810124e-04, -8.04495357e-06,  8.38647102e-05],
       [ 4.94136587e-02, -1.51565837e-06, -2.69689117e-05, ...,
         3.41140549e-05, -1.01994199e-04,  2.84221722e-04],
       ...,
       [-6.54428005e-02,  9.53386552e-06,  3.28887909e-05, ...,
         7.01069439e-05,  1.14341339e-04, -2.56687868e-04],
       [ 5.99011704e-02, -1.09144585e-05, -5.63615067e-05, ...,
         4.51343221e-05, -1.92771317e-04, -1.97840709e-04],
       [-4.54486720e-02, -1.79859289e-05, -1.64482790e-05, ...,
        -3.69285408e-06, -8.67230483e-05, -1.00684047e-04]], dtype=float32)>

In [37]:
hidden.weights

[<tf.Variable 'dense_14/kernel:0' shape=(784, 50) dtype=float32, numpy=
 array([[-0.01770425,  0.00407095,  0.07000803, ...,  0.08013842,
         -0.07911243,  0.01101536],
        [-0.01970632,  0.00456458,  0.05945834, ...,  0.08379038,
          0.07810465, -0.08296434],
        [ 0.056421  ,  0.07724804,  0.00417263, ...,  0.0619384 ,
          0.03587858, -0.04258172],
        ...,
        [-0.06198139, -0.07481034,  0.07102262, ...,  0.00600304,
         -0.04394611,  0.03386247],
        [ 0.0122844 ,  0.0843849 ,  0.05133539, ...,  0.03520263,
         -0.05640824,  0.07732203],
        [ 0.08064725,  0.07204448,  0.05897743, ..., -0.04463573,
          0.03161991,  0.06805336]], dtype=float32)>,
 <tf.Variable 'dense_14/bias:0' shape=(50,) dtype=float32, numpy=
 array([ 0.01858092, -0.00567263, -0.07340959, -0.03671593, -0.05480817,
        -0.15721531, -0.08629031, -0.04539269, -0.03571798, -0.15483859,
         0.05694963, -0.11505955, -0.06954357, -0.0231346 , -0.12895995,
